In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_validate
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, balanced_accuracy_score

import xgboost as xgb
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

In [22]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('final_proteins_dataframe.csv', index_col=0)
# df = pd.read_csv('proteins_dataframe.csv', index_col=0)
df

,pdb_id,chain_id,sequence,residue,residue_number,res_index,pdb_res_index,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y,U,avg_hydrophobicity,avg_hydrophilicity,avg_nci,avg_polarity,avg_polarizability,avg_sasa,avg_vol,hydrophobicity,hydrophilicity,NCI,polarity,polarizability,SASA,vol,is_charged,tm_segment
0,1vgo,A,DLLNDGRPETLWLGIGTLLMLIGTFYFIARGWGVTDKEAREYYAIT...,D,3,0,5,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.003333,0.433333,0.040025,9.400000,0.119333,1.595333,54.283333,-0.90,3.0,-0.023820,13.0,0.105,1.587,40.0,1,0
1,1vgo,A,DLLNDGRPETLWLGIGTLLMLIGTFYFIARGWGVTDKEAREYYAIT...,L,10,1,6,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,-0.502000,1.120000,0.056747,13.380000,0.201400,2.426400,86.140000,1.06,-1.8,0.051672,4.9,0.186,1.931,93.5,0,0
2,1vgo,A,DLLNDGRPETLWLGIGTLLMLIGTFYFIARGWGVTDKEAREYYAIT...,L,10,2,7,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,-0.107500,0.200000,0.149690,16.700000,0.304750,3.486000,131.525000,1.06,-1.8,0.051672,4.9,0.186,1.931,93.5,0,0
3,1vgo,A,DLLNDGRPETLWLGIGTLLMLIGTFYFIARGWGVTDKEAREYYAIT...,N,12,3,8,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,-1.656667,3.533333,0.161263,31.300000,0.412333,5.062000,166.600000,-0.78,0.2,0.005392,11.6,0.134,1.655,58.7,0,0
4,1vgo,A,DLLNDGRPETLWLGIGTLLMLIGTFYFIARGWGVTDKEAREYYAIT...,D,3,4,9,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,-3.675000,7.400000,0.326357,52.750000,0.658000,8.283000,251.950000,-0.90,3.0,-0.023820,13.0,0.105,1.587,40.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138182,7b0o,A,LYFQGHMDRLITLVVSYSIAFSIFALATMAVVYGKWLYYFEIDFLN...,V,18,211,206,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,-2.920000,2.650000,0.182683,41.500000,1.173500,11.068500,501.550000,1.08,-1.5,0.057004,5.9,0.140,1.645,71.5,0,0
138183,7b0o,A,LYFQGHMDRLITLVVSYSIAFSIFALATMAVVYGKWLYYFEIDFLN...,K,9,212,207,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,2.0,-2.300000,2.366667,0.104565,26.033333,0.720333,6.735333,303.200000,-1.50,3.0,0.017708,11.3,0.219,2.258,100.0,1,0
138184,7b0o,A,LYFQGHMDRLITLVVSYSIAFSIFALATMAVVYGKWLYYFEIDFLN...,K,9,213,208,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,3.0,-1.790000,2.350000,0.070333,17.975000,0.465750,4.459500,198.075000,-1.50,3.0,0.017708,11.3,0.219,2.258,100.0,1,0
138185,7b0o,A,LYFQGHMDRLITLVVSYSIAFSIFALATMAVVYGKWLYYFEIDFLN...,K,9,214,209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,4.0,-0.926000,1.280000,0.047549,12.280000,0.314400,3.055600,137.460000,-1.50,3.0,0.017708,11.3,0.219,2.258,100.0,1,0


In [23]:
X = df.iloc[:, 4:43]
X = X.drop(['pdb_res_index'], axis=1)
y = df['tm_segment']
X

# X = df.iloc[:, 4:15]
# X = X.drop(['pdb_res_index'], axis=1)
# y = df['tm_segment']
# X

,residue_number,res_index,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y,U,avg_hydrophobicity,avg_hydrophilicity,avg_nci,avg_polarity,avg_polarizability,avg_sasa,avg_vol,hydrophobicity,hydrophilicity,NCI,polarity,polarizability,SASA,vol,is_charged
0,3,0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.003333,0.433333,0.040025,9.400000,0.119333,1.595333,54.283333,-0.90,3.0,-0.023820,13.0,0.105,1.587,40.0,1
1,10,1,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,4.0,-0.502000,1.120000,0.056747,13.380000,0.201400,2.426400,86.140000,1.06,-1.8,0.051672,4.9,0.186,1.931,93.5,0
2,10,2,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,-0.107500,0.200000,0.149690,16.700000,0.304750,3.486000,131.525000,1.06,-1.8,0.051672,4.9,0.186,1.931,93.5,0
3,12,3,0.0,0.0,2.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,-1.656667,3.533333,0.161263,31.300000,0.412333,5.062000,166.600000,-0.78,0.2,0.005392,11.6,0.134,1.655,58.7,0
4,3,4,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,-3.675000,7.400000,0.326357,52.750000,0.658000,8.283000,251.950000,-0.90,3.0,-0.023820,13.0,0.105,1.587,40.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138182,18,211,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,2.0,1.0,-2.920000,2.650000,0.182683,41.500000,1.173500,11.068500,501.550000,1.08,-1.5,0.057004,5.9,0.140,1.645,71.5,0
138183,9,212,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,2.0,2.0,-2.300000,2.366667,0.104565,26.033333,0.720333,6.735333,303.200000,-1.50,3.0,0.017708,11.3,0.219,2.258,100.0,1
138184,9,213,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,1.0,3.0,-1.790000,2.350000,0.070333,17.975000,0.465750,4.459500,198.075000,-1.50,3.0,0.017708,11.3,0.219,2.258,100.0,1
138185,9,214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,4.0,-0.926000,1.280000,0.047549,12.280000,0.314400,3.055600,137.460000,-1.50,3.0,0.017708,11.3,0.219,2.258,100.0,1


In [7]:
# Data normilizations
scaler = StandardScaler()
scaler.fit(X)
scaled = scaler.fit_transform(X)
scaled_df = pd.DataFrame(scaled, columns=X.columns)
scaled_df

,residue_number,Hydrophobicity,Hydrophilicity,NCI,Polarity,Polarizability,SASA,V,is_charged
0,-1.268115,-1.235368,2.046999,-1.111856,2.103068,-0.561175,-0.316707,-0.789434,2.268825
1,-0.037428,0.827785,-0.777668,0.109571,-1.166392,0.308144,0.440488,0.677903,-0.440757
2,-0.037428,0.827785,-0.777668,0.109571,-1.166392,0.308144,0.440488,0.677903,-0.440757
3,0.314196,-1.109052,0.399277,-0.639219,1.537976,-0.249938,-0.167029,-0.276552,-0.440757
4,-1.268115,-1.235368,2.046999,-1.111856,2.103068,-0.561175,-0.316707,-0.789434,2.268825
...,...,...,...,...,...,...,...,...,...
138180,1.369070,0.848838,-0.601126,0.195840,-0.762755,-0.185544,-0.189040,0.074512,-0.440757
138181,-0.213241,-1.866945,2.046999,-0.439951,1.416885,0.662310,1.160264,0.856178,2.268825
138182,-0.213241,-1.866945,2.046999,-0.439951,1.416885,0.662310,1.160264,0.856178,2.268825
138183,-0.213241,-1.866945,2.046999,-0.439951,1.416885,0.662310,1.160264,0.856178,2.268825


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=False) # 7 indicates that always get the same split of data each time this example is executed

In [26]:
# fit model on training data
model = XGBClassifier(objective="binary:logistic", n_estimators = 100, learning_rate = 0.01, max_depth = 12)
model.fit(X_train, y_train)

# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

# evaluate predictions
accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

# evaluate predictions
accuracy_bl = balanced_accuracy_score(y_test, predictions)
print("Balanced accuracy: %.2f%%" % (accuracy_bl * 100.0))

roc_score = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
print("Roc: %.2f%%" % (roc_score* 100.0))

f1 = f1_score(y_test, predictions)
print("F1 score: %.2f%%" % (f1* 100.0))

Accuracy: 77.11%
Balanced accuracy: 76.89%
Roc: 84.11%
F1 score: 75.00%


In [11]:
inner_metric = 'balanced_accuracy'
outer_metrics = ['balanced_accuracy', 'average_precision', 'f1']

# configure the cross-validation procedure
cv_inner = StratifiedKFold(n_splits=3, shuffle=True)
cv_outer = StratifiedKFold(n_splits=10, shuffle=True)

model = XGBClassifier(objective="binary:logistic")

param_grid = {
  "max_depth": [1, 3, 7, 10],
  "n_estimators": [10, 500, 1000],
  "learning_rate": [0.01, 0.1, 0.2],
}

# define search
rf_grid = GridSearchCV(model, param_grid, scoring='balanced_accuracy', n_jobs=-1, cv=cv_inner, refit=True)

# execute the nested cross-validation
scores = cross_validate(rf_grid, scaled_df, y, scoring='balanced_accuracy', cv=cv_outer, n_jobs=-1)

#Best
rf_grid.fit(X, y)
# params[r] = rf_grid.best_params_
print("Best parameters: ", rf_grid.best_params_)

# scores[r] = rf_grid.best_score_
print("Best score: ", rf_grid.best_score_)

Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.6129532102797542
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.6129532386523935
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.6129532044164002
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.612953192615667
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.6129532610790335
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.6129532122198383
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.612953238518026
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.6129532072701179
Best parameters:  {'learning_rate': 0.01, 'max_depth': 1, 'n_estimators': 10}
Best score:  0.6129531919977071
Best paramet

In [ ]:
from sklearn.svm import SVC

svm = SVC(gamma='auto', probability=True)
svm.fit(X_train, y_train)

ypred = svm.predict(X_test)

# evaluate ypred
accuracy = accuracy_score(y_test.values, ypred)
print("Aaccuracy: %.2f%%" % (accuracy * 100.0))

# evaluate ypred
accuracy_bl = balanced_accuracy_score(y_test.values, ypred)
print("Balanced accuracy: %.2f%%" % (accuracy_bl * 100.0))

roc_score = roc_auc_score(y_test.values, svm.predict_proba(X_test)[:, 1])
print("Roc: %.2f%%" % (roc_score* 100.0))

f1 = f1_score(y_test.values, ypred)
print("F1 score: %.2f%%" % (f1* 100.0))

In [27]:
import lightgbm as lgb

train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

params = {
  "learning_rate": [0.01, 0.1, 0.15, 0.2, 0.3],
  "max_depth": [2,4,6,8,10,12,14],
  "num_leaves": [50,60,70,80,90]
}

model = lgb.LGBMClassifier(learning_rate=0.3, max_depth=12, num_leaves=70, num_iterations=500, objective='binary')

# num_round = 10
# lgb.cv(param, train_data, num_round, nfold=5)
model.fit(X_train,y_train, eval_set=[(X_test,y_test),(X_train,y_train)], verbose=20, eval_metric=['auc', 'logloss', 'average_precision'])

[20]	training's auc: 0.88775	training's binary_logloss: 0.424005	training's average_precision: 0.862172	valid_0's auc: 0.843349	valid_0's binary_logloss: 0.490033	valid_0's average_precision: 0.812319
[40]	training's auc: 0.907021	training's binary_logloss: 0.392026	training's average_precision: 0.884741	valid_0's auc: 0.839494	valid_0's binary_logloss: 0.496019	valid_0's average_precision: 0.805588
[60]	training's auc: 0.922899	training's binary_logloss: 0.363493	training's average_precision: 0.908182	valid_0's auc: 0.835846	valid_0's binary_logloss: 0.503054	valid_0's average_precision: 0.801696
[80]	training's auc: 0.936486	training's binary_logloss: 0.337851	training's average_precision: 0.923714	valid_0's auc: 0.831453	valid_0's binary_logloss: 0.511	valid_0's average_precision: 0.79737
[100]	training's auc: 0.94661	training's binary_logloss: 0.316526	training's average_precision: 0.936323	valid_0's auc: 0.829323	valid_0's binary_logloss: 0.51572	valid_0's average_precision: 0.796

LGBMClassifier(learning_rate=0.3, max_depth=12, num_iterations=500,
               num_leaves=70, objective='binary')

In [28]:
print('Training accuracy {:.4f}'.format(model.score(X_train,y_train)))
print('Testing accuracy {:.4f}'.format(model.score(X_test,y_test)))

Training accuracy 0.9732
Testing accuracy 0.7442
